In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install torch
!pip install transformers
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 40.1 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
import re
import torch
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from transformers import AutoModelForSequenceClassification, AdamW

In [3]:
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model = AutoModel.from_pretrained("monologg/kobert")

In [4]:
# 데이터셋 다운로드
train_data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t')
test_data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep='\t')

# Null 값 제거
train_data = train_data.dropna(how = 'any')
test_data = test_data.dropna(how = 'any')

# 정규 표현식을 이용한 한글 외 문자 제거
train_data['document'] = train_data['document'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
test_data['document'] = test_data['document'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))

def preprocess(data):
    sentences = data['document']
    labels = data['label'].values

    tokenized_sentences = []
    for sentence in sentences:
        tokenized_sentence = tokenizer.encode(sentence, truncation=True, padding='max_length', max_length=64)
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences, labels

train_inputs, train_labels = preprocess(train_data)
test_inputs, test_labels = preprocess(test_data)

from torch.utils.data import DataLoader, TensorDataset

BATCH_SIZE = 16

train_dataset = TensorDataset(torch.tensor(train_inputs), torch.tensor(train_labels))
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = TensorDataset(torch.tensor(test_inputs), torch.tensor(test_labels))
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# 수정된 코드.
from torch.cuda.amp import autocast, GradScaler

model = AutoModelForSequenceClassification.from_pretrained("monologg/kobert", num_labels=3)
optimizer = AdamW(model.parameters(), lr=1e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

EPOCHS = 5
GRADIENT_ACCUMULATION_STEPS = 2
scaler = GradScaler()

for epoch in range(EPOCHS):
    for i, batch in enumerate(train_loader):
        input_ids = batch[0].to(device)
        labels = batch[1].to(device)
        
        optimizer.zero_grad()
        
        with autocast():
            outputs = model(input_ids, labels=labels)
            loss = outputs.loss / GRADIENT_ACCUMULATION_STEPS
        
        scaler.scale(loss).backward()
        
        if (i + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()
        
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch[0].to(device)
            labels = batch[1].to(device)
            
            with autocast():
                outputs = model(input_ids)
                predictions = torch.argmax(outputs.logits, dim=-1)
                
            total += labels.size(0)
            correct += (predictions == labels).sum().item()

    accuracy = correct / total
    print("Epoch:", epoch + 1, "Accuracy:", accuracy)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 1 Accuracy: 0.49654979298757923
Epoch: 2 Accuracy: 0.5034502070124207


In [ ]:
# 시간 너무 오래걸림 일단 보류.

model = AutoModelForSequenceClassification.from_pretrained("monologg/kobert", num_labels=3)
optimizer = AdamW(model.parameters(), lr=1e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

EPOCHS = 5

for epoch in range(EPOCHS):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        labels = batch[1].to(device)
        outputs = model(input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch[0].to(device)
            labels = batch[1].to(device)
            outputs = model(input_ids)
            predictions = torch.argmax(outputs.logits, dim=-1)
            total += labels.size(0)
            correct += (predictions == labels).sum().item()

    accuracy = correct / total
    print("Epoch:", epoch + 1, "Accuracy:", accuracy)

In [ ]:
import torch

model.eval()

sentences = ["이 영화 정말 최고다!", "그냥 그런 영화였다.", "이 영화 개망했다..."]
labels = [1, 0, 2]

tokenized_sentences = []
for sentence in sentences:
    tokenized_sentence = tokenizer.encode(sentence, truncation=True, padding='max_length', max_length=64)
    tokenized_sentences.append(tokenized_sentence)

input_ids = torch.tensor(tokenized_sentences)
labels = torch.tensor(labels)

with torch.no_grad():
    outputs = model(input_ids)
    predictions = torch.argmax(outputs.logits, dim=-1)

print(predictions)